In [ ]:
import os
import warnings
import numpy as np
warnings.filterwarnings('ignore')
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    import matplotlib.pyplot as plt
    import seaborn as sns

    from simgen.project import Project
    import mbuild as mb
    import metamds as mds
    import mdtraj as md
    #from mdtraj.html import TrajectorySliderView
    import nglview
    from ipywidgets.widgets import Box

%matplotlib inline
!rm -rf simgen_output/ *.top *.gro

In [ ]:
github_project_url = 'https://github.com/iModels/demos/demos/monolayer/simgen_project'
from simgen.mdsblocks.editor import Editor
editor = Editor('place_here_your_github_token', github_project_url)
editor.show()

In [ ]:
def build_monolayer(chain_length, n_molecules, **kwargs):
    from mbuild.examples import AlkaneMonolayer
    pattern = mb.Random2DPattern(n_molecules)
    monolayer = AlkaneMonolayer(pattern, tile_x=1, tile_y=1, chain_length=chain_length)
    monolayer.name = 'alkane_n-{}_l-{}'.format(n_molecules, chain_length)
    mb.translate(monolayer, [0, 0, 2])
    box = monolayer.boundingbox
    monolayer.periodicity += np.array([0, 0, 5 * box.lengths[2]])
    return monolayer

In [ ]:
def create_run_script(**parameters):
    project = Project(github_project_url+'/online_project.yaml')

    return project.render_tasks('prg', output_dir='./', inject_dict=parameters)

In [ ]:
# Initialize simulation with a template and some metadata
sim = mds.Simulation(name='monolayer', 
                     template=create_run_script,
                     input_dir='static_input_files',
                     output_dir='simgen_output')

chain_lengths = [8, 12, 16, 20]
for length in chain_lengths:
    parameters = {'chain_length': length,
                  'n_molecules': 100,
                  'forcefield': 'OPLS-aa'}
    
    compound = build_monolayer(**parameters)
    parameters['compound'] = compound
    parameters['system_name'] = compound.name
    
    # Parameterize our simulation template
    sim.parametrize(**parameters)

In [ ]:
# Run
REMOTE_EXECUTION=True
if REMOTE_EXECUTION:
    sim.execute_all(hostname='rahman.vuse.vanderbilt.edu', username='imodels')
else:
    sim.execute_all()

In [ ]:
if REMOTE_EXECUTION:
    sim.sync_all()

In [ ]:
task_1 = next(sim.tasks())
trj_path = os.path.join(task_1.output_dir, 'nvt.xtc')
top_path = os.path.join(task_1.output_dir, 'em.gro')
traj = md.load(trj_path, top=top_path)
print(traj)

In [ ]:
t = nglview.MDTrajTrajectory(traj)
w = nglview.NGLWidget(t)
w.representations = [{'type':'ball+stick'}]
Box(children=(w,))

In [ ]:
average_S2 = list()
for task in sim.tasks():
    # Load up the trajectory
    trj_path = os.path.join(task.output_dir, 'nvt.xtc')
    top_path = os.path.join(task.output_dir, 'em.gro')
    traj = md.load(trj_path, top=top_path)
    
    # Nematic order parameter
    atoms_per_chain = int((traj.n_atoms - 1800) / 100)
    chain_indices = [[n+x for x in range(atoms_per_chain)] 
                     for n in range(1800, traj.n_atoms, atoms_per_chain)]
    s2 = md.compute_nematic_order(traj, indices=chain_indices)
    average_S2.append(np.mean(s2))
    
    
plt.plot(chain_lengths, average_S2)
plt.xlabel('chain length (# carbon atoms)')
plt.ylabel('Mean S2')